### Import Libraries

In [1]:
from lib2to3.fixes.fix_input import context

import pandas as pd
import json
import great_expectations as gx
import sys
import re
import os
from great_expectations.checkpoint.actions import EmailAction
from great_expectations.expectations.expectation import ColumnMapExpectation
from great_expectations.execution_engine.pandas_execution_engine import PandasExecutionEngine
from great_expectations.execution_engine.sqlalchemy_execution_engine import SqlAlchemyExecutionEngine
from great_expectations.expectations.metrics import ColumnMapMetricProvider
from great_expectations.expectations.metrics.map_metric_provider import (column_map_metric_provider, column_condition_partial)

C:\Users\TechTeam-08\AppData\Local\Temp\ipykernel_6480\628268446.py:1: DeprecationWarning: lib2to3 package is deprecated and may not be able to parse Python 3.10+
  from lib2to3.fixes.fix_input import context


### Creating GX Context

In [ ]:
context = gx.get_context(mode="ephemeral")

### Creating Datasource

In [ ]:
data_source_name = "emp data"
data_source = context.sources.add_pandas(name=data_source_name)

### Creating DataAsset

In [ ]:
data_asset_name = "emp_asset"
data_asset = data_source.add_dataframe_asset(name=data_asset_name)

### Creating Batch Definition

In [ ]:
batch_definition_name = "emp_batch"
batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)

### Creating Metric for PAN Validation

In [ ]:
class ColumnValuesToMatchPanFormat(ColumnMapMetricProvider):
    condition_metric_name = "column_values.match_pan_format"

    @column_condition_partial(engine=PandasExecutionEngine)
    def _pandas(cls, column, **kwargs):
        pan_regex = r"^[A-Z]{5}[0-9]{4}[A-Z]{1}$"
        return column.apply(lambda x: bool(re.match(pan_regex, str(x))))

    @column_condition_partial(engine=SqlAlchemyExecutionEngine)
    def _sqlalchemy(cls, column, **kwargs):
        pan_regex = r"^[A-Z]{5}[0-9]{4}[A-Z]{1}$"
        return column.like(f'[A-Z]{5}[0-9]{4}[A-Z]{1}')

### Create Custom Expectation

In [ ]:
class ExpectColumnValuesToMatchPanFormat(ColumnMapExpectation):
    map_metric = "column_values.match_pan_format"
    success_keys = ("mostly",)
    default_kwarg_values = {
        "mostly": 1.0,
        "result_format": "BASIC",
        "include_config": True
    }

### Creating Expectation

In [ ]:
expectation_pan = ExpectColumnValuesToMatchPanFormat(column="pan")

### Creating ExpectationSuite and Adding Expectations

In [ ]:
expectation_suite_name = "emp_suite"
expectation_suite_ref = gx.ExpectationSuite(name=expectation_suite_name)
expectation_suite = context.suites.add(expectation_suite_ref)

In [ ]:
expectation_suite.add_expectation(expectation_pan)

### Creating Validation Definition

In [ ]:
validation_def_name = "emp_data_validation"
validation_definition_ref = gx.ValidationDefinition( data=batch_definition,
                                                    suite=expectation_suite,
                                                    name=validation_def_name)

In [ ]:
validation_definition = context.validations.add(validation_definition_ref)

### Creating Send Email Action

In [4]:
# Opening JSON file
f = open('C:\\Users\\TechTeam-08\\PycharmProjects\\Essential Data Quality Principles, Implement Testing with Python and Great Expectations Framework\\emp_details.json')

# returns JSON object as a dictionary
data = json.load(f)

# Iterating through the json list
for i in data['email']:
    print(i)


KeyboardInterrupt



In [ ]:
email_action = EmailAction(
    name="send_email",
    notify_on = "all",
    smtp_address="smtp.gmail.com",
    smtp_port=587,
    sender_password =